<a href="https://colab.research.google.com/github/Mariana-zy/SyS-2025-2S/blob/main/Taller2/Punto_1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ejercicio 5:** Diseñe e implemente un detector de género musical, que permita diferenciar entre dos tipos de género. Para ello, construya una base de datos con al menos 50 registros por género musical, de 5 segundos cada uno, y luego implemente un código que permita recibir un nuevo registro músical de 5 segundos para detectar el género de la canción a partir de los vecinos más cercanos según la distancia Euclídea entre la magnitud del espectro de Fourier del nuevo registro (ver [clasificador KNN](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)), y las magnitudes de los espectros de la base de datos construida (Ver [Cuaderno YouTube Download/Detector](https://github.com/amalvarezme/SenalesSistemas/blob/master/3_SerieyTransformadaFourier/youtube_detector.ipynb)).

In [8]:
#cargar datos desde drive acceso libre
#1UVHl8wyAafw1kgvke9IMi7fwUNv8ZswA
FILEID = "1UVHl8wyAafw1kgvke9IMi7fwUNv8ZswA" #"1DxI5wQpqEWksw2BqJnG7n0IgNQ_xByX-"

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O  canciones.xlsx && rm -rf /tmp/cookies.txt
#!unzip -o codigos.zip
!dir

--2025-11-11 04:37:44--  https://docs.google.com/uc?export=download&confirm=&id=1UVHl8wyAafw1kgvke9IMi7fwUNv8ZswA
Resolving docs.google.com (docs.google.com)... 142.251.12.101, 142.251.12.100, 142.251.12.139, ...
Connecting to docs.google.com (docs.google.com)|142.251.12.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1UVHl8wyAafw1kgvke9IMi7fwUNv8ZswA&export=download [following]
--2025-11-11 04:37:44--  https://drive.usercontent.google.com/download?id=1UVHl8wyAafw1kgvke9IMi7fwUNv8ZswA&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.68.132, 2404:6800:4003:c02::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.68.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9263 (9.0K) [application/octet-stream]
Saving to: ‘canciones.xlsx’

canciones.xlsx      100%[===================>]   9.05K  --.-KB/s 

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file_ = 'canciones.xlsx'#leer archivo xlsx con link, band, type
X  = pd.read_excel(file_)
X#imprimir filas iniciales

,link,band,type,type_num
0,https://www.youtube.com/watch?v=WdoXZf-FZyA&ab...,megadeth,metal,1
1,https://www.youtube.com/watch?v=LOXrjUuXEn0&ab...,megadeth,metal,1
2,https://www.youtube.com/watch?v=CD-E-LDc384&ab...,metallica,metal,1
3,https://www.youtube.com/watch?v=boanuwUMNNQ&ab...,metallica,metal,1
4,https://www.youtube.com/watch?v=nspxAG12Cpc&ab...,tool,metal,1
5,https://www.youtube.com/watch?v=CehYA3omb5o&ab...,tool,metal,1
6,https://www.youtube.com/watch?v=lGEoJLIIuow&ab...,amonamarth,metal,1
7,https://www.youtube.com/watch?v=edBYB1VCV0k&ab...,amonamarth,metal,1
8,https://www.youtube.com/watch?v=lfreJbC1YOY&ab...,jbalvin,reggaeton,2
9,https://youtu.be/KHAgoT4FZbc,jbalvin,reggaeton,2


In [10]:
# --- INSTALACIÓN DE LIBRERÍAS NECESARIAS ---
!pip install -U yt-dlp
!apt-get install -y ffmpeg
!pip install soundfile

import yt_dlp as youtube_dl
import subprocess
import os

# --- FUNCIÓN PARA DESCARGAR Y CONVERTIR A MP3 ---
def download_ytvid_as_mp3(video_url, name):
    options = {
        'format': 'bestaudio/best',
        'keepvideo': False,
        'outtmpl': f'{name}.mp3',
    }
    with youtube_dl.YoutubeDL(options) as ydl:
        try:
            ydl.download([video_url])
            print(f"✅ Download complete: {name}.mp3")
        except Exception as e:
            print(f"❌ Error descargando {video_url}: {e}")


#funcion para descargar mp3 desde youtube
def download_ytvid_as_mp3(video_url,name):
    #video_url = input("enter url of youtube video:")
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':f'{name}.mp3',
    }
    if cookies:
        options['cookiefile'] = None
        options['cookiejar'] = cookies;

    with youtube_dl.YoutubeDL(options) as ydl:
        try:
            video_info = ydl.extract_info(video_url, download=False)
            ydl.download([video_info['webpage_url']])
        except Exception as e:
            print(f"Error descargando {video_url}: {e}")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [11]:
import subprocess
import os

#crear carpeta con resultados
try:
  os.mkdir('results')
except:
  print("Carpeta results ya existe")

#recorrer excel con videos
N, P = X.shape
Ns = N * 5 #cantidad de segmentos por cancion

os.makedirs('results', exist_ok=True)

for n in range(N):
    print(f"video {n+1} de {N}")
    print(f"link: {X.loc[n,'link']}\n")
    print(f"band: {X.loc[n,'band']}\n")
    print(f"type: {X.loc[n,'type']}\n")
    #ruta video n-th
    name_ = 'results/'+X.loc[n,'band']+"_"+str(n)+"_"+str(X.loc[n,'type_num']) # #video+nombre+tipo de genero musical
    #descargar mp3 desde youtube
    try: # Added try block to catch download errors
        download_ytvid_as_mp3(X.loc[n,'link'],name_)
        #convertir a .wav
        subprocess.call(['ffmpeg', '-y', '-i', name_+'.mp3', name_+'.wav'])

    except youtube_dl.utils.DownloadError as e: # Catch the specific DownloadError
        print(f"Skipping video {X.loc[n,'link']} due to download error: {e}") # Print a message and skip

Carpeta results ya existe
video 1 de 16
link: https://www.youtube.com/watch?v=WdoXZf-FZyA&ab_channel=Megadeth-Topic

band: megadeth

type: metal

[youtube] Extracting URL: https://www.youtube.com/watch?v=WdoXZf-FZyA&ab_channel=Megadeth-Topic
[youtube] WdoXZf-FZyA: Downloading webpage


[youtube] WdoXZf-FZyA: Downloading android sdkless player API JSON
[youtube] WdoXZf-FZyA: Downloading web safari player API JSON


ERROR: [youtube] WdoXZf-FZyA: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Error descargando https://www.youtube.com/watch?v=WdoXZf-FZyA&ab_channel=Megadeth-Topic: ERROR: [youtube] WdoXZf-FZyA: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
video 2 de 16
link: https://www.youtube.com/watch?v=LOXrjUuXEn0&ab_channel=Megadeth-Topic

band: megadeth

type: metal

[youtube] Extracting URL: https://www.youtube.com/watch?v=LOXrjUuXEn0&ab_channel=Megadeth-Topic
[youtube] LOXrjUuXEn0: Downloading webpage


[youtube] LOXrjUuXEn0: Downloading android sdkless player API JSON
[youtube] LOXrjUuXEn0: Downloading web safari player API JSON


...

ERROR: [youtube] lixcxBrXR2w: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Error descargando https://www.youtube.com/watch?v=lixcxBrXR2w&ab_channel=FeidVEVO: ERROR: [youtube] lixcxBrXR2w: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
video 16 de 16
link: https://www.youtube.com/watch?v=mLCqa3OpspU&ab_channel=Feid

band: feid

type: reggaeton

[youtube] Extracting URL: https://www.youtube.com/watch?v=mLCqa3OpspU&ab_channel=Feid
[youtube] mLCqa3OpspU: Downloading webpage


[youtube] mLCqa3OpspU: Downloading android sdkless player API JSON
[youtube] mLCqa3OpspU: Downloading web safari player API JSON


ERROR: [youtube] mLCqa3OpspU: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Error descargando https://www.youtube.com/watch?v=mLCqa3OpspU&ab_channel=Feid: ERROR: [youtube] mLCqa3OpspU: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


In [12]:
#cargar .wavs y partir audios
#lista archivos .wav
path = 'results/'
wav_files = [f for f in os.listdir(path) if f.endswith('.wav')]
wav_files

[]

In [13]:
import soundfile as sf # para instalar pip install soundfile
from scipy.signal import resample_poly
import numpy as np
import os

#leer archivos y crear np.array audios
fs = 48000
tl = np.array([30,40,50,60,70,80]) #puntos lectura
ts = 5 #t segmento
# The total number of samples for each segment at the target fs
segment_length_fs = int(ts * fs)
Ns = len(wav_files)*len(tl) #cantidad segmentos
# x_t should be initialized with the shape corresponding to the target fs
x_t = np.zeros((Ns, segment_length_fs, 2)) #Ns segmentos, cantidad de muestras, 2 canales (stereo)
label = np.zeros((Ns,1)) #vector tipo de genero
type_c = X['type'].unique()
name_c = []
#leer archivos wav
i = 0
for name in wav_files:#lectura audio .wav
    x, fs_i = sf.read(path+name)
    # Ensure x is at least 2D (for stereo)

    if x.ndim == 1:
        x = np.expand_dims(x, axis=1) # Add a channel dimension if mono
    if x.shape[1] == 1: # If still mono after expand_dims
        x = np.repeat(x, 2, axis=1) # Convert mono to stereo by repeating the channel

    # Check if the audio file has enough data for the requested segments
    if len(x) < int(fs_i * (tl[-1] + ts)):
        print(f"Warning: File {name} is too short for all requested segments. Skipping some segments.")
        valid_tl = tl[tl + ts <= len(x)/fs_i]
    else:
        valid_tl = tl

    for ti in valid_tl: #segmentos de tiempo
        print(name,'x',x.shape,'fs actual:',fs_i)

        # Slice based on the original sampling rate fs_i
        start_sample = int(fs_i * ti)
        end_sample = int(fs_i * (ti + ts))
        xc = x[start_sample:end_sample, :]

        # Check if the sliced segment has enough data for resampling
        # This can happen if the calculated end_sample exceeds the actual audio length
        if xc.shape[0] < int(ts * fs_i):
            print(f"Warning: Segment from {ti}s to {ti+ts}s in {name} is shorter than expected. Skipping.")
            continue # Skip this segment

        if fs_i != fs:
            # Resample using resample_poly
            # The output length of resample_poly is len(xc) * up / down
            # We want the resampled length to be exactly segment_length_fs
            # So we need to calculate up and down such that len(xc) * up / down = segment_length_fs
            # Or simply use the desired number of samples directly if supported by resample_poly
            # resample_poly can take 'nyq' or 'kaiser_beta' arguments for filter design,
            # but for simple resampling to a target number of points, a direct approach is needed.
            # A simpler resample function might be better, or calculate up/down differently.

            # Let's use a method that targets the desired number of samples directly.
            # scipy.signal.resample can do this, but it uses FFT and might not be ideal.
            # Let's adjust the resampling calculation with resample_poly to ensure the correct output size.
            # Target output size is segment_length_fs.
            # Current size is xc.shape[0].
            # We want xc.shape[0] * (up / down) = segment_length_fs
            # Let's find up and down that approximate fs/fs_i
            # A more robust way is to use a library that handles resampling to a specific length.
            # For now, let's ensure the resampled array is truncated or padded if necessary,
            # although it's better to get the resampling right.

            # A common approach for arbitrary resampling is
            # import librosa
            # xc_resampled = librosa.resample(xc, orig_sr=fs_i, target_sr=fs)
            # However, this outputs a 1D array for mono. Need to handle stereo.

            # Let's stick with resample_poly but ensure the output is the correct size.
            # If resampling makes it slightly longer or shorter, we'll handle it.
            gcd_val = np.gcd(fs, fs_i)
            up_val = fs // gcd_val
            down_val = fs_i // gcd_val

            # Resample each channel separately
            xc_resampled_ch1 = resample_poly(xc[:, 0], up=up_val, down=down_val)
            xc_resampled_ch2 = resample_poly(xc[:, 1], up=up_val, down=down_val)

            # Combine channels
            xc_resampled = np.stack((xc_resampled_ch1, xc_resampled_ch2), axis=-1)

            # Ensure the resampled segment has the target length (segment_length_fs)
            if xc_resampled.shape[0] > segment_length_fs:
                xc_resampled = xc_resampled[:segment_length_fs, :]
            elif xc_resampled.shape[0] < segment_length_fs:
                # Pad with zeros if shorter
                padding = np.zeros((segment_length_fs - xc_resampled.shape[0], 2))
                xc_resampled = np.vstack((xc_resampled, padding))

            xc = xc_resampled # Use the resampled array
            print(f"Resampled to shape: {xc.shape}")

        # Now xc should have the shape (segment_length_fs, 2) which is (240000, 2)
        if xc.shape[0] != segment_length_fs:
             print(f"Error: Resampled segment has incorrect length {xc.shape[0]}. Expected {segment_length_fs}. Skipping.")
             continue # Skip this segment if resampling failed to produce correct length


        x_t[i] = xc
        # The label indexing seems to assume the order of wav_files matches the order in X
        # and the type_num is the last digit before '.wav' and is 1 or 2.
        # This might be fragile. Consider using the 'type_num' column from X directly
        # based on the filename or a mapping.
        # For now, sticking to the original logic for the label.
        try:
             # Extract type_num from the filename
             parts = name.split('_')
             type_num_str = parts[-1].split('.')[0]
             label[i] = int(type_num_str)
        except (IndexError, ValueError):
             print(f"Warning: Could not extract type_num from filename {name}. Setting label to 0.")
             label[i] = 0 # Assign a default or error label

        # The name_c logic also seems to assume the filename format.
        # Sticking to original logic.
        name_c += [name[:-4]] # Remove '.wav' from the end


        print(f"{i} lectura: {name}; segundo {ti}:{ti+ts}; tipo música: {type_c[int(label[i])-1]}")
        i+=1

# After the loop, if some segments were skipped, the actual number of populated segments in x_t and label
# might be less than Ns. We should truncate these arrays to the actual number of populated segments.
# Ns was calculated based on the initial assumption of processing all segments.
# Let's re-calculate the effective Ns (number of processed segments).
effective_Ns = i
x_t = x_t[:effective_Ns]
label = label[:effective_Ns]
name_c = name_c[:effective_Ns]

x_t.shape

(0, 240000, 2)

In [14]:
from IPython.display import Audio #reproducir segmento
i = 0
Audio(x_t[i].T,rate=fs)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
#calculo de fourier
vf = np.fft.rfftfreq(x_t.shape[1],1/fs) #calculo vector de frecuencias
Xw = np.fft.rfft(x_t,axis=1).mean(axis=-1) #transformada rapida de Fourier para señal Real a lo largo del tiempo (axis=1) y se promedian los dos canales
Xw.shape

In [ ]:
#grafica tiempo y fourier
plt.plot(np.arange(0,ts,1/fs),x_t.mean(axis=-1).T) #se promedian los dos canales stereo
plt.xlabel('t[s]')
plt.ylabel('x(t)')
plt.show()

In [ ]:
plt.plot(vf,abs(Xw).T)
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)|')
plt.show()

In [ ]:
#se normalizan espectros entre 0 y 1 para evitar inconsistencias por ampliltudes máximas
from sklearn.preprocessing import MinMaxScaler
sca = MinMaxScaler()
Xw_ = sca.fit_transform(abs(Xw).T).T

plt.plot(vf,Xw_.T)
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)|')
plt.show()

#en dB
plt.plot(vf,(20*np.log10(Xw_+1e-10)).T) # se suma 1e-10 para evitar discontinuidad del log
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)| dB')
plt.show()

In [ ]:
from umap import UMAP
#visualización de datos
n_neighbors = int(2*np.sqrt(Xw_.shape[0]))
sca_ = MinMaxScaler()

red_ = UMAP(n_components=2,n_neighbors=n_neighbors)
X_2D = sca_.fit_transform(red_.fit_transform(Xw_))

In [ ]:
#graficar separabilidad 2D
plt.scatter(X_2D[:,0],X_2D[:,1],c=label)
plt.colorbar()
plt.show()

color_ = ["b","y"]
#nombre cancion
plt.scatter(X_2D[:,0],X_2D[:,1],c=label,s=1)
for i, tex in enumerate(name_c):
    #print(f"{i} {tex}")
    plt.text(X_2D[i,0]*1.025,X_2D[i,1]*1.025, tex[:-2]+"_"+str(i), fontsize=6,color=color_[int(label[i]-1)])

plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.show()

In [ ]:
#reproducir audio
i = 1
Audio(x_t[i].T,rate=fs)

In [ ]:
# Predicción del género musical sobre espectro de Fourier

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Crear una instancia del clasificador RandomForest
model = KNeighborsClassifier(n_neighbors=5)

# Realizar la validación cruzada
# Xw_ son tus características y label son tus etiquetas
# cv define el número folds para la validación cruzada
scores = cross_val_score(model, Xw_, label.ravel(), cv=5,scoring='accuracy')

# Imprimir los resultados de la validación cruzada
print("Acierto para cada Fold:", scores)
print("Acierto promedio:", scores.mean())
print("Desviación estándar del Acierto:", scores.std())

#entrenar modelo para predecir nuevas canciones
model.fit(Xw_, label.ravel())

In [ ]:
import joblib
#guardar modelo
try:
  os.mkdir('modelo')
except:
  print("Carpeta modelo ya existe")

filename_ = 'modelo/reggaeton_vs_metal'
model_ ={'Xw_':Xw_, 'label' : label, 'name_c' : name_c, 'vf':vf,'fs':fs, 'modelo':model,'type':X['type'].unique()}
joblib.dump(model_,filename_+".pkl")


In [ ]:
#descargar modelo
#guardar resultados
from google.colab import files
from datetime import date, datetime
import shutil
#guardar resultados
namefile = str(datetime.now().strftime("%Y_%m_%d_%H_%M_%d"))+'modelo'
shutil.make_archive(namefile, 'zip', 'modelo')
files.download(namefile+'.zip')

#el archivo .zip puede cargarse en drive y utilizarse en otro cuaderno para detectar género musical de nuevos segmentos

In [ ]:
#cargar modelo
my_model_loaded = joblib.load(filename_+".pkl")
my_model_loaded.keys()

In [ ]:
#evaluar para un segmento en fft normalizado

pred_ = my_model_loaded['modelo'].predict(Xw_[0].reshape(1,-1))[0]
print('Etiqueta estimada: ',my_model_loaded['type'][int(pred_-1)])
print('Etiqueta orignal: ',my_model_loaded['type'][int(my_model_loaded['label'][0][0])-1])